# Intro to Backends

One note from the Author - Backends are still experimental, while completely functional, there may be some bugs. I would encourage you to test out these Backends, and please file an issue with any inconsistency you find or thoughts/questions you have.

Best - kcaverly

## What is a 'Backend'

Backends, fundamentally take in a `Signature`, `Example` and misc arguments and produce `Completions`.
`Completions` are objects, which return Language Model generations as structured `Signature` aligned objects.

Historically, DSPy, has hidden most Backend functionality into a singular backend, exposing only `LM` as the medium to augment generation-only functionality. Prompt templates, chat messaging, and tool use has had to fit into the existing "backend". Opening up these Backends to the user directly allows greater functionality, robustness, and transparency.

Publically, there are three pieces of functionality that Backends expose:

**1. Prompt Generation**
- Given a Signature and Examples, generate a Prompt template and Language Model Arguments.

**2. Generation**
- Given a Prompt/Messages & Additional arguments, generate text from a Language Model.

**3. Extraction**
- Given the output of a Language Model, extract the Signature field values, generated by the Language Model.

All three pieces, are tightly coupled, and make up a single Backend.

Privately, there are three pieces of functionality that Backends inherit:

**1. Caching**
- Traditionally, each LM module built had to have caching built into the individual model.
- All Backends, immediately enable caching out of the box. If you pass the same arguments into a 'generate' call in the Backend, you will return the same outputs.

**2. Tracing**
- We now track standardized inputs and outputs, for all Backends.
- This allows us to get greater transparency into the individual inputs/outputs to each call.

**3. Incomplete Retries**
- In DSPy currently, during generation, if an incomplete Completion is provided, the Generation is retried until a complete Example is provided.
- All backends, inherit this functionality, and will retry an individual completion a few times until a valid Example is generated.

This simplifies and standardizes previous DSPy functionality.

To start we have created a `TextBackend`, which mimics existing DSPy functionality, and should provide . This API allows us to expand into novel Backends as well, such as a `JSONBackend` (for JSON Mode models), `ToolBackend` (for Too Use), and even bespoke backends leveraging community projects such as `OutlinesBackend` or `InstructorBackend`.

## How are 'Backend's used?

Previously all Language Model functionality is expressed in the `dsp.modules`.  
Most of the existing language model functionality should ship out of the box with the new `TextBackend`.



In [ ]:
# For example, to initialize GPT-3 as the primary Language Model you would do this:
import dspy
from dspy import OpenAI

lm = OpenAI(model="gpt-3.5-turbo-instruct")

dspy.settings.configure(lm=lm)

# If on the other hand, you want to use Cohere's Command-R, you would do this:
from dspy import Cohere

lm = Cohere(model="command-r")
dspy.settings.configure(lm=lm)

Backends are configured in the same way as language models have been historically. And All Predict modules, will just leverage a backend if initialized. To mirror the behaviour above, with the new Backends, you can use the new `TextBackend`. This backend, leverages [LiteLLM](https://docs.litellm.ai/docs/providers) behind the scenes, which enables a wide variety of Language Model providers out of the box, without additional maintenance work needed on the DSPy side.

In [ ]:
# To enable GPT-3 as the primary language model with the new backends, you would do this:
import dspy
from dspy.modeling import TextBackend

backend = TextBackend(model="gpt-3.5-turbo-instruct")
dspy.settings.configure(backend=backend)

# And if you wanted to change this to cohere, you would simply do the following:
backend = TextBackend(model="command-r")
dspy.settings.configure(backend=backend)

With one string, and no additional dependencies, we've opened up 100+ language models and provided a much greater level of robustness and flexibility, while reducing maintenance burden on the team.

## How are 'Backends' expressed in code?

At a high level the private logic for a Backend and abstract methods are illustrated in the `BaseBackend` object.  
The public api looks something like the below:

In [ ]:
import typing as t
from abc import abstractmethod
from dspy import Signature, Example

class BaseBackend:
    ...
    
    @abstractmethod
    def prepare_request(self, signature: Signature, example: Example, config: dict, **kwargs) -> dict:
        """Given a Signature, Example, and Config kwargs, provide a dictionary of arguments for the Backend."""
        ...

    @abstractmethod
    def process_response(
        self,
        signature: Signature,
        example: Example,
        response: t.Any,
        input_kwargs: dict,
        **kwargs,
    ) -> Completions:
        """Given a Signature, Example, and Generated Output, process generations and return completions."""
        ...

    @abstractmethod
    def make_request(self, **kwargs) -> t.Any:
        ...


This api, allows us to express all Language Model interactions as a single, composable, but extensible API.  
If you are interested in how more of this functionality is provided, I would encourage you to take a look at the `TextBackend` and `JSONBackend` objects.